## Imports

In [1]:
import numpy as np
import pandas as pd
import os
from preprocessing import Preprocessor
from dataset_balancing import Balance

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM, Input
from tensorflow.keras.metrics import Precision

In [3]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, log_loss, roc_curve, fbeta_score

## Data preprocessing

In [4]:
# Load training and test sets
df_train = pd.read_json('datas/training_set.json')
df_test = pd.read_json('datas/testing_set.json')

print(f"Train shape : {df_train.shape}")
print(f"Test shape : {df_test.shape}")

Train shape : (6035, 2)
Test shape : (1065, 2)


In [5]:
# Balance the train set
train_balance = Balance(df_train)
train_balance.process_balance('./preprocessed_data/train_balanced')

In [6]:
# Preprocess the data
train_preprocessor = Preprocessor(train_balance.df_dataset)
train_preprocessor.preprocess('./preprocessed_data/train_nlp_preprocessed')
test_preprocessor = Preprocessor(df_test)
test_preprocessor.preprocess('./preprocessed_data/test_nlp_preprocessed')

(15997, 300)
(15997, 8)


C:\Users\Lenovo\Documents\Paul\Travail\ENSEIRB-MATMECA\3A\Ingenierie_logicielle_IA\projet_industrialisation_ia\preprocessing.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array(data)


(1065, 300)
(1065, 8)


In [7]:
train_preprocessor.df_dataset.head()

,sentence,intent
0,"[1.9918094, -2.0211177, -3.1467276, -0.7597667...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
1,"[0.37102285, -0.25975516, -2.5644374, 1.239701...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2,"[0.5238122, -1.4262346, -0.8189889, 0.9439444,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
3,"[2.6711895, -3.8386414, -3.3684235, 0.06278875...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
4,"[0.16812666, -1.4507266, -3.699288, -0.9399957...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"


In [28]:
x_train = np.array(list(train_preprocessor.df_dataset['sentence']))
y_train = np.array(list(train_preprocessor.df_dataset['intent']))
x_test = np.array(list(test_preprocessor.df_dataset['sentence']))
y_test = np.array(list(test_preprocessor.df_dataset['intent']))

print("x_train.shape = ", x_train.shape)
print("y_train.shape = ", y_train.shape)
print("x_test.shape = ", x_test.shape)
print("y_test.shape = ", y_test.shape)

x_train.shape =  (15997, 300)
y_train.shape =  (15997, 8)
x_test.shape =  (1065, 300)
y_test.shape =  (1065, 8)


In [29]:
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print("x_train.shape = ", x_train.shape)
print("x_test.shape = ", x_test.shape)

x_train.shape =  (15997, 1, 300)
x_test.shape =  (1065, 1, 300)


## Model creation and training

In [30]:
model = Sequential()
#model.add(Input(shape=x_train[0].shape))
model.add(Bidirectional(LSTM(64), input_shape=x_train[0].shape))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 520       
Total params: 199,816
Trainable params: 199,816
Non-trainable params: 0
________________________________________________

In [31]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=[Precision()]
)

history = model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1)

Epoch 1/20
125/125 [==============================] - 4s 4ms/step - loss: 1.8591 - precision_2: 0.6630
Epoch 2/20
125/125 [==============================] - 0s 4ms/step - loss: 0.8779 - precision_2: 0.8431
Epoch 3/20
125/125 [==============================] - 0s 4ms/step - loss: 0.5602 - precision_2: 0.8866
Epoch 4/20
125/125 [==============================] - 0s 4ms/step - loss: 0.4165 - precision_2: 0.9178
Epoch 5/20
125/125 [==============================] - 0s 4ms/step - loss: 0.3229 - precision_2: 0.9336
Epoch 6/20
125/125 [==============================] - 0s 4ms/step - loss: 0.2571 - precision_2: 0.9445
Epoch 7/20
125/125 [==============================] - 0s 4ms/step - loss: 0.2161 - precision_2: 0.9540
Epoch 8/20
125/125 [==============================] - 0s 4ms/step - loss: 0.1720 - precision_2: 0.9639
Epoch 9/20
125/125 [==============================] - 0s 4ms/step - loss: 0.1499 - precision_2: 0.9682
Epoch 10/20
125/125 [==============================] - 1s 4ms/step - loss

In [32]:
model.save('./models/model_v1')

INFO:tensorflow:Assets written to: ./models/model_v1\assets


INFO:tensorflow:Assets written to: ./models/model_v1\assets


In [33]:
score = model.evaluate(x_test, y_test)

34/34 [==============================] - 2s 2ms/step - loss: 1.1721 - precision_2: 0.7850


In [34]:
print("Test loss =", score[0])
print("Test precision =", score[1])

Test loss = 1.17210054397583
Test precision = 0.7850288152694702


In [35]:
y_pred = model.predict(x_test)
print(y_pred.shape)
print(y_pred[0])
print(y_test[0])

(1065, 8)
[3.8509662e-10 1.1290101e-02 2.0123975e-08 2.3965072e-06 1.3060239e-01
 8.5810167e-01 4.3734389e-07 2.9576945e-06]
[0. 1. 0. 0. 0. 0. 0. 0.]


In [36]:
y_test_true = list(map(lambda x: np.argmax(x), y_test))
y_test_pred = list(map(lambda x: np.argmax(x), y_pred))
print(len(y_test_true))
print(len(y_test_pred))

1065
1065


In [37]:
print(y_test_true[:10])
print(y_test_pred[:10])

[1, 1, 1, 4, 3, 1, 0, 1, 1, 1]
[5, 1, 1, 4, 3, 1, 0, 1, 1, 1]


In [38]:
fbeta = fbeta_score(y_test_true, y_test_pred, beta=0.5, labels=list(range(8)), average='weighted')
print(fbeta)

0.8004185925692892
